In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split

NBA_learn_df=pd.read_csv("../Resources/NBA_learning.csv")
NBA_learn_df

Using TensorFlow backend.


,Unnamed: 0,Year,Age,G,GS,MP,PTS,PER,TS%,FTr,FG,FGA,AST,PF,height,weight,Players ok,Played_Years
0,0,1986,22.0,82,1.0,1542.0,521,11.8,0.564,0.430,209,388,54,229,203,106,0,0
1,1,1987,23.0,79,72.0,2240.0,852,15.7,0.599,0.480,316,587,84,171,203,106,0,1
2,2,1988,24.0,82,64.0,2636.0,937,14.5,0.581,0.592,322,640,93,204,203,106,0,2
3,3,1989,25.0,82,82.0,2510.0,1088,17.8,0.594,0.474,401,758,103,172,203,106,0,3
4,4,1990,26.0,82,82.0,2709.0,1061,14.7,0.548,0.459,385,806,90,207,203,106,0,4
5,5,1991,27.0,82,21.0,2164.0,750,13.8,0.556,0.557,258,542,71,117,203,106,0,5
6,6,1992,28.0,82,53.0,2902.0,1116,16.7,0.556,0.569,382,803,117,141,203,106,0,6
7,7,1993,29.0,82,55.0,2819.0,1051,16.3,0.603,0.531,379,706,116,149,203,106,0,7
8,8,1994,30.0,82,55.0,2825.0,1204,17.0,0.555,0.391,465,926,137,142,203,106,0,8
9,9,1995,31.0,82,52.0,2687.0,916,14.2,0.596,0.556,311,617,127,146,203,106,0,9


In [2]:
NBA_learn_df.sample(frac=1).reset_index(drop=True)

,Unnamed: 0,Year,Age,G,GS,MP,PTS,PER,TS%,FTr,FG,FGA,AST,PF,height,weight,Players ok,Played_Years
0,17617,1995,30.0,80,18.0,2070.0,419,13.1,0.496,0.146,166,397,421,275,196,88,0,8
1,14565,1993,31.0,63,0.0,730.0,237,10.7,0.515,0.183,104,213,34,137,208,111,0,12
2,3941,2008,31.0,78,78.0,2522.0,1324,23.6,0.619,0.498,393,877,529,130,190,91,0,12
3,11478,1977,23.0,27,NaN,164.0,66,10.1,0.487,0.339,26,59,10,26,196,88,0,1
4,16099,1997,26.0,31,0.0,153.0,62,11.1,0.448,0.350,25,60,1,17,206,95,0,1
5,22087,1973,26.0,51,NaN,1603.0,490,9.3,0.470,0.330,185,455,138,117,190,79,0,4
6,6056,2004,22.0,80,78.0,2444.0,909,11.6,0.473,0.234,376,871,158,140,196,95,0,3
7,21958,1975,27.0,46,NaN,1748.0,719,16.4,0.535,0.352,280,582,99,175,201,97,0,4
8,2879,2005,25.0,68,68.0,1865.0,637,16.5,0.588,0.611,239,427,57,218,213,121,0,3
9,2445,1962,23.0,21,NaN,212.0,86,14.3,0.502,0.289,34,76,49,31,185,77,0,2


In [3]:
X=NBA_learn_df.drop(columns=["Unnamed: 0","Players ok"])
y=NBA_learn_df['Players ok']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [4]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [5]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [6]:
mod1 = Sequential() 
mod1.add(Dense(50, activation='relu',input_shape=(16,)))
mod1.add(Dropout(0.3))
mod1.add(Dense(100, activation='relu'))
#mod1.add(Dropout(0.3))
mod1.add(Dense(50, activation='relu'))

mod1.add(Dense(1, activation="sigmoid"))

mod1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                850       
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 11,051
Trainable params: 11,051
Non-trainable params: 0
_________________________________________________________________


In [7]:
mod1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='NBA.h5', monitor='loss',verbose=0, save_best_only='min', period=1)

h = mod1.fit(X_train, y_train, batch_size=1000, epochs=100, verbose=1, validation_data = (X_test,y_test),callbacks=[checkpointer])

Train on 18468 samples, validate on 6156 samples
Epoch 1/100
18468/18468 [==============================] - 1s 36us/step - loss: 0.7821 - accuracy: 0.9955 - val_loss: 0.6012 - val_accuracy: 0.9971
Epoch 2/100
18468/18468 [==============================] - 0s 6us/step - loss: 0.5543 - accuracy: 0.9980 - val_loss: 0.4983 - val_accuracy: 0.9971
Epoch 3/100
18468/18468 [==============================] - 0s 6us/step - loss: 0.4471 - accuracy: 0.9980 - val_loss: 0.3870 - val_accuracy: 0.9971
Epoch 4/100
18468/18468 [==============================] - 0s 7us/step - loss: 0.3342 - accuracy: 0.9980 - val_loss: 0.2755 - val_accuracy: 0.9971
Epoch 5/100
18468/18468 [==============================] - 0s 6us/step - loss: 0.2286 - accuracy: 0.9980 - val_loss: 0.1808 - val_accuracy: 0.9971
Epoch 6/100
18468/18468 [==============================] - 0s 7us/step - loss: 0.1462 - accuracy: 0.9980 - val_loss: 0.1149 - val_accuracy: 0.9971
Epoch 7/100
18468/18468 [==============================] - 0s 6us/st

18468/18468 [==============================] - 0s 6us/step - loss: 0.0145 - accuracy: 0.9980 - val_loss: 0.0202 - val_accuracy: 0.9971
Epoch 57/100
18468/18468 [==============================] - 0s 6us/step - loss: 0.0145 - accuracy: 0.9980 - val_loss: 0.0202 - val_accuracy: 0.9971
Epoch 58/100
18468/18468 [==============================] - 0s 6us/step - loss: 0.0145 - accuracy: 0.9980 - val_loss: 0.0202 - val_accuracy: 0.9971
Epoch 59/100
18468/18468 [==============================] - 0s 6us/step - loss: 0.0145 - accuracy: 0.9980 - val_loss: 0.0202 - val_accuracy: 0.9971
Epoch 60/100
18468/18468 [==============================] - 0s 6us/step - loss: 0.0144 - accuracy: 0.9980 - val_loss: 0.0202 - val_accuracy: 0.9971
Epoch 61/100
18468/18468 [==============================] - 0s 6us/step - loss: 0.0144 - accuracy: 0.9980 - val_loss: 0.0201 - val_accuracy: 0.9971
Epoch 62/100
18468/18468 [==============================] - 0s 7us/step - loss: 0.0144 - accuracy: 0.9980 - val_loss: 0.0201 

In [8]:
model_loss, model_accuracy = mod1.evaluate(
    X_test_scaled, encoded_y_test, verbose=0)
print("Normal Neural Network - Loss:" +str(model_loss)+", Accuracy: "+str(model_accuracy))

Normal Neural Network - Loss:0.020177502875428712, Accuracy: 0.9970760345458984


In [9]:
encoded_predictions = mod1.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

C:\Users\Joshua Castillo\AppData\Local\conda\conda\envs\actumlogos-gpu\lib\site-packages\sklearn\preprocessing\label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
print("Predicted classes:"+ str(prediction_labels))
print("Actual Labels: "+str(list(y_test[:10])))

Predicted classes:[0 0 0 0 0 0 0 0 0 0]
Actual Labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
NBA_learn_df=NBA_learn_df.dropna()
X=NBA_learn_df.drop(columns=["Unnamed: 0","PER"])
y=NBA_learn_df['PER']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_scaler = MinMaxScaler().fit(y_train.reshape(-1, 1))
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

AttributeError: 'Series' object has no attribute 'reshape'

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(X_train["Players ok"])
encoded_X_train = label_encoder.transform(X_train["Players ok"])
encoded_X_test = label_encoder.transform(X_test["Players ok"])

In [ ]:
y_train#.reshape(-1, 1)

In [ ]:
mod2 = Sequential() 
mod2.add(Dense(50, activation='relu',input_shape=(16,)))
#mod2.add(Dropout(0.3))
mod2.add(Dense(100, activation='relu'))
#mod2.add(Dropout(0.3))
mod2.add(Dense(50, activation='relu'))

mod2.add(Dense(1, activation="relu"))

mod2.summary()

In [ ]:
mod2.compile(loss="mse", optimizer='adam',metrics=['accuracy'])#, metrics=['accuracy']

checkpointer = ModelCheckpoint(filepath='NBA2.h5', monitor='loss',verbose=0, save_best_only='min', period=1)

h = mod2.fit(X_train, y_train, batch_size=1000, epochs=100, verbose=1, validation_data = (X_test,y_test),callbacks=[checkpointer])